In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [46]:
def load_data() -> pd.DataFrame:
    dataset = "country_vaccination_stats.csv"
    data = pd.read_csv(dataset)
    return data

In [47]:
def get_minimum_vaccination(data: pd.DataFrame, country: str) -> float:
    country_data = data[data['country'] == country]
    value = country_data['daily_vaccinations'].min()
    if pd.isna(value):
        return 0
    else:
        return value

In [48]:
def fill_missing_data(pd_data: pd.DataFrame) -> pd.DataFrame:
    for country in pd_data['country'].unique():
        country_data = pd_data[pd_data['country'] == country]
        for index, row in country_data.iterrows():
            if pd.isna(row['daily_vaccinations']):
                pd_data.at[index, 'daily_vaccinations'] = get_minimum_vaccination(pd_data, country) 
    return pd_data

# Example usage
data = load_data()
filled_data = fill_missing_data(data)
print(filled_data)

        country        date  daily_vaccinations  \
0     Argentina  12/29/2020              6483.0   
1     Argentina  12/30/2020             15656.0   
2     Argentina  12/31/2020             15656.0   
3     Argentina    1/1/2021             11070.0   
4     Argentina    1/2/2021              8776.0   
...         ...         ...                 ...   
1497      Wales   1/20/2021             11105.0   
1498      Wales   1/21/2021             12318.0   
1499      Wales   1/22/2021             15148.0   
1500      Wales   1/23/2021             17371.0   
1501      Wales   1/24/2021             17063.0   

                                 vaccines  
0                               Sputnik V  
1                               Sputnik V  
2                               Sputnik V  
3                               Sputnik V  
4                               Sputnik V  
...                                   ...  
1497  Oxford/AstraZeneca, Pfizer/BioNTech  
1498  Oxford/AstraZeneca, Pfizer/Bi